Simulate 100 scenarios with only one edge  
do full abba to see in eachone how many false positive I get  
do depuring abba to see how many false positives I get  
do depuring abba (only share ancestry test) to see how many false positives I get  

Same with 0 edges  
Same with 2 edges  

Compare them!

In [1]:
import ipyrad.analysis as ipa
import ipcoal
import toytree
import toyplot
import pandas as pd
from ipyrad.analysis.baba21 import Drawing

In [2]:
import random


def generate_tests_from_names(sources, targets_raw, outgroup, no_repeat=True):
    
    if type(sources) is not list:
        sources = [sources]
    
    if type(outgroup) is not list:
        outgroup = [outgroup]
    
    #Declare empty result
    tests =[]
    
    for source in sources:
        #If in sources are the outgroup skip it
        if source in outgroup:
            continue
        
        
        #New unlinked targets list every loop
        targets = targets_raw.copy()
            
        #Remove source and outgroups in case they are repeated in the targets list
        if source in targets:
            targets.remove(source)
        for i in outgroup:
            if i in targets:
                targets.remove(i) 

        #declare empty lists 
        included = []
        

        #Iterate over all targets
        for i, _ in enumerate(targets):
            #empty temp p variables
            p1 = [] 
            p2 = []

           #only do following if target was not already used (not in included)
            if targets[i] not in included:
                #use the first target as p1
                p1 = [targets[i]]
                #mark it is used appending to the list
                if no_repeat: included.append(targets[i])
                #avoid out of boundary errors
                if i < len(targets)-1:
                    #use the next target as p2
                    if targets[i+1] not in included:
                        p2 = [targets[i+1]]
                        #mask it as used
                        if no_repeat: included.append(targets[i+1])
                else:
                    #in case it is the last element (for odd number of targets, use a random one (but the current) as p2)
                    targets.remove(targets[i])
                    p2 =  [random.choice(targets)]

                #append test to the return list
                tests.append({'p1': p1, 'p2': p2, 'p3': [source], 'p4': outgroup})
                
    return tests #return list of dictionaries with every test

In [3]:
def get_significant_donee(df_result, test):
    if df_result["D"].values[0] > 0:
            significative_donee = test["p2"]
    elif df_result["D"].values[0] < 0:
        significative_donee = test["p1"]
    return significative_donee

In [4]:
def run_tests_depuring(data, tests, tree, zscoreTH=2.5, distant=True, verbose=False):
#v4
    tests_performed_dict = {}
    truepositives_network = []
    falsepositives_network = []
    n_tests = 0
    n_fp = 0
    
    #Do test by test
    for i in tests:
        if verbose: print("\n*** Testing:", i, "***")
        donor = i["p3"]
        outgroup = i["p4"]
        donees = [*i["p1"], *i["p2"]]
        
        
        #Any test is saved in a dict with the significative donee as value by donors, here I create donor dict
        if str(*donor) not in tests_performed_dict:
            tests_performed_dict[str(*donor)] = {}
        
        
        #Check if the test was already performed, get the significative donee and continue to other test
        if str(sorted(donees)) in tests_performed_dict[str(*donor)].keys():
            if verbose: print("test already performed (skipped) previous significative donee: ", tests_performed_dict[str(*donor)][str(sorted(donees))])
            continue
        

        #Save DF of resulst
        n_tests += 1
        try:
            df_result = data.run_test(i, nboots=100, quiet=True)
        except:
            continue
        

       
        
        
        #If test is significant
        if df_result["Z"].values[0] > zscoreTH:

            #Get if ABBA or BABA is the significant usind D
            significative_donee = get_significant_donee(df_result, i)

            #Update tests dict
            tests_performed_dict[str(*donor)][str(sorted(donees))] = str(*significative_donee)


            ## Check all phylo neighborhood to see if the significant is due to shared ancestry

            #Get sisters in the tree of significant donee
            sisters_significant_donee = tree.get_tip_labels(tree.idx_dict[tree.get_mrca_idx_from_tip_labels(significative_donee)].get_ancestors()[0].idx) 

            #Remove donor, sig_donee and outgroup from sisters list to avoid test donor-donor or donor-outgroup, donee-donee
            for ele in donor:
                if ele in sisters_significant_donee:
                    sisters_significant_donee.remove(ele)        
            for ele in outgroup:
                if ele in sisters_significant_donee:
                    sisters_significant_donee.remove(ele)
            for ele in significative_donee:
                if ele in sisters_significant_donee:
                    sisters_significant_donee.remove(ele)

            if verbose: print("donor: ", donor)
            if verbose: print("significative donee: ", significative_donee)
            
            
            in_clade_significants = []
            
            ## Test shared ancestry
            #Do nested test having two fixed elements, donor and one donee (being this one the significant_donee)
            #Assumption: if a vs b is significant, that significancy will maintain if we do a test involving
            #a vs b vs b_sister. If b significancy is lost, return it as false positive.
            if len(sisters_significant_donee) > 0:
                
                
                if verbose: print("--- Testing against sisters: ", sisters_significant_donee, "---")
                
                #Test against all sisters
                for sister in sisters_significant_donee:
                    s_sd = str(sorted([sister, *significative_donee]))
                    
                    

                    #Check if the test was already performed to skip it
                    if s_sd not in tests_performed_dict[str(*donor)].keys():
                        # Create the test
                        in_clade_test = {"p1":significative_donee,"p2":[sister],"p3":donor,"p4":outgroup}
                        
                        # Do a baba for the test
                        n_tests += 1
                        try:
                            in_clade_df_result = data.run_test(in_clade_test, nboots=100, quiet=True)
                        except:
                            continue
                        
                        
                        # If donee 2 is significant, save in performed test and add this result to the clade
                        if in_clade_df_result["Z"].values[0] > zscoreTH:
                            in_clade_sd = get_significant_donee(in_clade_df_result, in_clade_test)
#                             tests_performed_dict[str(*donor)][s_sd] = str(*in_clade_sd) #ToDo, this registry can include untested significants
                            
                            #Add to a list of significants in the clade, if more than one is in this list shared ancestry could be true
                            if in_clade_sd not in in_clade_significants:
                                in_clade_significants.append(str(*in_clade_sd))
                                          
                            if verbose: print("- significative donee found for", s_sd, ": ", str(*in_clade_sd))
                        else:
                            in_clade_sd = None
                            tests_performed_dict[str(*donor)][s_sd] = None
                            if verbose: print("- no significative donee found in", s_sd)
                        
                    else:
                        #If test was already done, just copy the result
                        in_clade_sd = tests_performed_dict[str(*donor)][s_sd]
                        if verbose: print("- test already performed (skipped) " + s_sd + ". Previous significative donee: ", tests_performed_dict[str(*donor)][s_sd])
                        #If result is different to None add it to in_clade_significants for further interpretation
                        if in_clade_sd:
                            if in_clade_sd not in in_clade_significants:
                                in_clade_significants.append(in_clade_sd)
            
            
            
            
            
            else:
                #If no more sisters in the clade because all of them were included in the main test.
                #For now it is assumed as false positive. ToDo: explore more about it.
#                 if verbose: print("All sisters already in the test")
                if verbose: print("Shared ancestry or too close related, assumed as false positive")
#                 if str(*significative_donee) not in in_clade_significants:
#                     in_clade_significants.append(str(*significative_donee))
                    
                    
            ## Decision maker
            # Based on in_clade_significants results in previous steps (a.k.a. true positives for verification) do               
            if verbose: print("in_clade_significants", in_clade_significants)
                
            if in_clade_significants:
                if str(*significative_donee) in in_clade_significants:
                    if len(in_clade_significants) > 1:
                        if verbose: print("shared true")
                        #ToDo: maybe return ancestral node instead of tip in tree. Do some simulations with this scenario
                    else:
                        if verbose: print("true positive for verification")
                    
#                    
                    
                    ## Distance test: despite there is some true positive, it still may be false negative. For example
                    # tests where the pair compared are very distant from the donor, so any minimal
                    # allele frequency common in both may be give this false result
                    # Using sister as outgroups and put the outgroup as pair, I induce a max distancing distorion
                    # if it pass, true positive is verified, if not, it may be a artifact caused by distance
                    #distant test to reduce false positives caused by distant samples
                    if distant:
                        sisters_donor = tree.get_tip_labels(tree.idx_dict[tree.get_mrca_idx_from_tip_labels(donor)].get_ancestors()[0].idx) 
                        #remove donor from sister group
                        if str(*donor) in sisters_donor:
                            sisters_donor.remove(str(*donor))
                        
                        #remove significant donee from sister group
#                         if str(*in_clade_significants) in sisters_donor:
#                             sisters_donor.remove(str(*in_clade_significants))
                        #correct bug in case in_clade_significants is more than one elment
                        for ics in in_clade_significants:
                            if ics in sisters_donor:
                                sisters_donor.remove(ics)


                        #if still something is in sisters_donor group do distant test
                        if sisters_donor:

                            #Create distant test
                            distant_test = {"p1":outgroup,"p2":in_clade_significants,"p3":donor,"p4":sisters_donor}
                            if verbose: print(">>> distant test to verify significant donee:", distant_test, "<<<")

                            #Do distant test
                            n_tests += 1
                            try:
                                distant_df_result = data.run_test(distant_test, nboots=100, quiet=True)
                            except:
#                                 print("fail in distant")
                                truepositives_network.append((*donor, *in_clade_significants))
                                break



                           
                            if distant_df_result["Z"].values[0] > zscoreTH:
                                distant_sd = get_significant_donee(distant_df_result, distant_test)
                                if distant_sd == in_clade_significants:
                                    if (*donor, *in_clade_significants) not in truepositives_network:
                                        truepositives_network.append((*donor, *in_clade_significants))
                                    if verbose: print("true verified", distant_sd)
                                else:
                                    if verbose: print("falsed by distance", distant_sd)
                                    n_fp += 1
                            else:
                                if verbose: print("no signficant in this test true positive rejected")
                                n_fp += 1

                        # when no sisters to tests distance bias do
                        else:
                            if verbose: print("imposible to do distant tests") #significant donnee assummed as false positive")
    #                         if (*donor, *in_clade_significants) not in truepositives_network:
    #                             truepositives_network.append((*donor, *in_clade_significants))
                    else:
                         truepositives_network.append((*donor, *in_clade_significants))
                        
                



                else:
                    if verbose: print("false but others in the clade?")
                    n_fp += 1
                    
                    
            else:
                if verbose: print("false")
                n_fp += 1
                    
            
            
        
        
        #No significant result in test
        else:
            if verbose: print("No significant donee in test")
            tests_performed_dict[str(*donor)][str(sorted(donees))] = None
    
    if verbose: print ("\nNumber of tests performed:" + str(n_tests))
    if verbose: print ("False positives depured:" + str(n_fp))
#     if verbose: print (tests_performed_dict)
    return truepositives_network

In [ ]:
#Create scenarios

tips = 20
scenarios = 100
name = "scene_1"

# generate a balance tree
tree = toytree.rtree.baltree(ntips=tips, treeheight=10e6)

for scenario in range(scenarios):
   
    #select two random tips
    random_tips = random.sample(range(tips-1), 2)

    # create a simulation model for this tree/network: (src, dest, time-prop., admix-prop.)
    model = ipcoal.Model(tree=tree, nsamples=2, Ne=4e5, admixture_edges=(random_tips[0], random_tips[1], 0.5, 0.15))

    # simulate N loci
    model.sim_loci(nloci=3000, nsites=50)

    # drop 50% as missing
    model.apply_missing_mask(0.5)

    # write result to a database file
    model.write_snps_to_hdf5(name=f"{name}/test-baba-miss50_{scenario}_{random_tips[0]}to{random_tips[1]}", diploid=True)

In [ ]:
%%capture full
#First tests using complete depur

import os
arr = os.listdir("/home/carlos/AutoABBA/scene_1/")

tips=20
tree = toytree.rtree.baltree(ntips=tips, treeheight=10e6)

final_result = []

for hdf5 in arr: 
    
    trueSource, trueTarget = hdf5.split("_")[2].split(".")[0].split("to")
    test_id = hdf5.split("_")[1]
    
    print (test_id)
    
    baba = ipa.baba21(f"/home/carlos/AutoABBA/scene_1/{hdf5}")
    source = tree.get_tip_labels()
    targets = tree.get_tip_labels()
    outgroup = "r19"

    tests = generate_tests_from_names(source, targets, outgroup, no_repeat=False)


    result = run_tests_depuring(baba, tests, tree, verbose=False, distant=True)

#     print([test_id, trueSource, trueTarget, result])
#     final_result.append([test_id, trueSource, trueTarget, result])
#     print(result)
#     with open("fulldepur.txt", "a") as f:
#         f.write(str([test_id, trueSource, trueTarget, result])+ "\n")
    file = open("fulldepur.txt", "a")
    file.write(str([test_id, [trueSource, trueTarget], result])+ "\n")
    file.close()

In [13]:
cap_out.stdout

'18\n70\n80\n61\n42\n52\n31\n65\n50\n8\n19\n67\n15\n0\n11\n2\n76\n73\n'

In [ ]:
tips=20
tree = toytree.rtree.baltree(ntips=tips, treeheight=10e6)
source = tree.get_tip_labels()
targets = tree.get_tip_labels()
outgroup = "r19"

tests = generate_tests_from_names(source, targets, outgroup, no_repeat=False)
baba = ipa.baba21(f"/home/carlos/AutoABBA/scene_1/test-baba-miss50_73_16to2.snps.hdf5")
result = run_tests_depuring(baba, tests, tree, verbose=False, distant=True)

In [ ]:
print(1)